In [1]:
from tqdm import tqdm

## 1. Pre-train on CIFAR-10

In [3]:
'''Train CIFAR10 with PyTorch.'''
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
from tqdm import tqdm

from models import *

os.environ["CUDA_VISIBLE_DEVICES"]='4,5'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 250], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('./checkpoint'):
            os.mkdir('./checkpoint')
        torch.save(state, './checkpoint/googLeNet.cifar10.t7')
        best_acc = acc


for epoch in range(start_epoch, 351):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

==> Building model..

Epoch: 0


  0%|          | 0/391 [00:00<?, ?it/s]

Saving..

Epoch: 1


...


Epoch: 53


  0%|          | 0/391 [00:00<?, ?it/s]


Epoch: 54


Acc: 88.687%:  81%|████████▏ | 318/391 [00:57<00:13,  5.52it/s]Process Process-123:
Process Process-124:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/amy/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/home/amy/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/

KeyboardInterrupt: 

## Train on CINIC-10

- train: train set
- valid: valid set

In [1]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import PIL

from models import *

os.environ["CUDA_VISIBLE_DEVICES"]='4,5'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='../data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

testset = torchvision.datasets.ImageFolder(root='../data/CINIC-10/valid/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('./checkpoint/googLeNet.cifar10.t7')
net.load_state_dict(checkpoint['net'])
#start_epoch = checkpoint['epoch']
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 250, 400], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('./checkpoint'):
            os.mkdir('./checkpoint')
        torch.save(state, './checkpoint/googLeNet.cinic10.0.t7')
        best_acc = acc


for epoch in range(start_epoch, 601):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

==> Preparing data..
==> Building model..
==> Resuming from checkpoint..


  0%|          | 0/704 [00:00<?, ?it/s]


Epoch: 0


  0%|          | 0/704 [00:00<?, ?it/s]

Saving..

Epoch: 1


...

  0%|          | 0/704 [00:00<?, ?it/s]


Epoch: 599


  0%|          | 0/704 [00:00<?, ?it/s]


Epoch: 600


Acc: 85.783%: 100%|██████████| 900/900 [00:48<00:00, 18.38it/s]


## Train on full CINIC-10

- train: train set + valid set
- valid: test set

In [ ]:
'''Train CINIC10 with PyTorch.'''
from __future__ import print_function
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
import PIL

import os
import argparse

from models import *

os.environ["CUDA_VISIBLE_DEVICES"]='4,5'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])

trainset = torchvision.datasets.ImageFolder(root='../data/CINIC-10/train/', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

validset = torchvision.datasets.ImageFolder(root='../data/CINIC-10/valid/', transform=transform_train)
validloader = torch.utils.data.DataLoader(validset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='../data/CINIC-10/test/', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)



classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)


net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load('./checkpoint/googLeNet.cinic10.1.t7')
net.load_state_dict(checkpoint['net'])
#start_epoch = checkpoint['epoch']
best_acc = checkpoint['acc']
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 250, 450], gamma=0.1)

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(trainloader)+len(validloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)
        for batch_idx, (inputs, targets) in enumerate(validloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
            pbar.update(1)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with tqdm(total=len(testloader)) as pbar:
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_description('Acc: %.3f%%' % (100.*correct/total))
                pbar.update(1)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('./checkpoint'):
            os.mkdir('./checkpoint')
        torch.save(state, './checkpoint/googLeNet.cinic10.1.t7')
        best_acc = acc

for epoch in range(start_epoch, 601):
    scheduler.step()
    train(epoch)
    if epoch % 10 == 0:
        test(epoch)

## Export Model

In [1]:
_name = 'cinic10.1'
src_path = './checkpoint/googLeNet.{}.t7'.format(_name)
dest_path = './checkpoint/googLeNet.{}.pth'.format(_name)

In [2]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import os
import argparse

from models import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Load checkpoint.
print('==> Resuming from checkpoint..')
checkpoint = torch.load(src_path)
net.load_state_dict(checkpoint['net'])

==> Building model..
==> Resuming from checkpoint..


In [3]:
net = net.module

In [4]:
state = {
    'net': net.state_dict(),
    'acc': checkpoint['acc'],
    'epoch': checkpoint['epoch'],
}

torch.save(state, dest_path)

In [5]:
# Export model
print('==> Exporting model..')
dummy_input = Variable(torch.randn(128, 3, 32, 32), requires_grad=False).cuda()
torch.onnx.export(net, dummy_input, "./checkpoint/googLeNet.{}.onnx".format(_name), verbose=True)

==> Exporting model..
graph(%0 : Float(128, 3, 32, 32)
      %1 : Float(192, 3, 3, 3)
      %2 : Float(192)
      %3 : Float(192)
      %4 : Float(192)
      %5 : Float(192)
      %6 : Float(192)
      %7 : Float(64, 192, 1, 1)
      %8 : Float(64)
      %9 : Float(64)
      %10 : Float(64)
      %11 : Float(64)
      %12 : Float(64)
      %13 : Float(96, 192, 1, 1)
      %14 : Float(96)
      %15 : Float(96)
      %16 : Float(96)
      %17 : Float(96)
      %18 : Float(96)
      %19 : Float(128, 96, 3, 3)
      %20 : Float(128)
      %21 : Float(128)
      %22 : Float(128)
      %23 : Float(128)
      %24 : Float(128)
      %25 : Float(16, 192, 1, 1)
      %26 : Float(16)
      %27 : Float(16)
      %28 : Float(16)
      %29 : Float(16)
      %30 : Float(16)
      %31 : Float(32, 16, 3, 3)
      %32 : Float(32)
      %33 : Float(32)
      %34 : Float(32)
      %35 : Float(32)
      %36 : Float(32)
      %37 : Float(32, 32, 3, 3)
      %38 : Float(32)
      %39 : Float(32)
      %40 : 

## Testing

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import sys
data_dir = "/tmp/work/data/CINIC-10/test/"
assert data_dir is not None, "No data directory"

from models import *
checkpoint = torch.load('../mobileNetV2.cinic10.0.pth')
model = MobileNetV2()
model.load_state_dict(checkpoint['net'])
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def inference(model, testloader):
    total = 0
    correct = 0
    model.to(device)
    model = torch.nn.DataParallel(model)
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets.cuda()).sum().item()
    acc = 100.*correct/total
    print(acc)
    return acc

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.47889522, 0.47227842, 0.43047404],  std=[0.24205776, 0.23828046, 0.25874835]),
])
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
inference(model, testloader)